# Capacity Planning with Monte Carlo Simulation

This notebook demonstrates Monte Carlo simulation for semiconductor fab capacity planning.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

## 1. Define Machine Configuration

In [ ]:
# Machine configurations
machines = [
    {"name": "LITHO-01", "base_throughput": 100, "efficiency_mean": 0.95, "efficiency_std": 0.03, "downtime_prob": 0.02},
    {"name": "LITHO-02", "base_throughput": 100, "efficiency_mean": 0.88, "efficiency_std": 0.04, "downtime_prob": 0.03},
    {"name": "ETCH-01", "base_throughput": 120, "efficiency_mean": 0.92, "efficiency_std": 0.03, "downtime_prob": 0.02},
    {"name": "ETCH-02", "base_throughput": 120, "efficiency_mean": 0.85, "efficiency_std": 0.05, "downtime_prob": 0.04},
    {"name": "DEP-01", "base_throughput": 110, "efficiency_mean": 0.90, "efficiency_std": 0.04, "downtime_prob": 0.03},
    {"name": "DEP-02", "base_throughput": 110, "efficiency_mean": 0.87, "efficiency_std": 0.04, "downtime_prob": 0.03},
    {"name": "INSP-01", "base_throughput": 80, "efficiency_mean": 0.93, "efficiency_std": 0.02, "downtime_prob": 0.01},
    {"name": "CLEAN-01", "base_throughput": 90, "efficiency_mean": 0.89, "efficiency_std": 0.03, "downtime_prob": 0.02},
]

## 2. Run Monte Carlo Simulation

In [ ]:
def run_simulation(machines, time_horizon_days=30, n_simulations=10000):
    results = []
    
    for _ in range(n_simulations):
        total_output = 0
        
        for day in range(time_horizon_days):
            day_output = 0
            
            for machine in machines:
                # Check for downtime
                is_down = np.random.random() < machine["downtime_prob"]
                
                if not is_down:
                    # Normal operation
                    efficiency = np.random.normal(
                        machine["efficiency_mean"],
                        machine["efficiency_std"]
                    )
                    efficiency = np.clip(efficiency, 0.3, 1.0)
                    
                    daily_output = machine["base_throughput"] * efficiency * 24
                    daily_output *= np.random.normal(1.0, 0.02)
                    day_output += daily_output
            
            total_output += day_output
        
        results.append(total_output)
    
    return np.array(results)

# Run simulation
results = run_simulation(machines, n_simulations=5000)

## 3. Analyze Results

In [ ]:
# Statistics
print(f"Mean Throughput: {np.mean(results):.0f} wafers")
print(f"Std Dev: {np.std(results):.0f} wafers")
print(f"P5: {np.percentile(results, 5):.0f} wafers")
print(f"P95: {np.percentile(results, 95):.0f} wafers")
print(f"P99: {np.percentile(results, 99):.0f} wafers")

In [ ]:
# Distribution plot
plt.figure(figsize=(10, 6))
plt.hist(results, bins=50, edgecolor='black', alpha=0.7)
plt.axvline(np.mean(results), color='red', linestyle='--', label='Mean')
plt.axvline(np.percentile(results, 5), color='orange', linestyle='--', label='P5')
plt.axvline(np.percentile(results, 95), color='orange', linestyle='--', label='P95')
plt.xlabel('Total Throughput (wafers)')
plt.ylabel('Frequency')
plt.title('Monte Carlo Simulation: 30-Day Throughput Distribution')
plt.legend()
plt.show()

## 4. Bottleneck Analysis

In [ ]:
# Calculate expected contribution per machine
time_horizon_days = 30
contributions = []

for machine in machines:
    contrib = machine["base_throughput"] * machine["efficiency_mean"] * 24 * time_horizon_days
    contributions.append({
        "name": machine["name"],
        "expected_contribution": contrib
    })

contributions_df = pd.DataFrame(contributions)
contributions_df = contributions_df.sort_values("expected_contribution")

# Plot
plt.figure(figsize=(10, 6))
plt.barh(contributions_df["name"], contributions_df["expected_contribution"])
plt.xlabel('Expected Contribution (wafers)')
plt.title('Machine Contribution Analysis')
plt.show()

print(f"\nTop Bottleneck: {contributions_df.iloc[0]['name']}")